In [136]:
import requests
import random
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime , timedelta
import re



path = "//Users//hajiaga//Desktop//DataScience//chromedriver"

driver = webdriver.Chrome()

driver.implicitly_wait(5)


In [137]:
def azetoengdate(string):

    gun = string[:2]

    ay = str(re.findall(r'\D+', string)).replace(" ","").lower()

    if ay == "yanvar":
        numay = "01"
    elif ay == "fevral":
        numay = "02"
    elif ay == "mart":
        numay = "03"
    elif ay == "aprel":
        numay = "04"
    elif ay == "may":
        numay = "05"
    elif ay == "iyun":
        numay = "06"
    elif ay == "iyul":
        numay = "07"
    elif ay == "avqust":
        numay = "08"
    elif ay == "sentyabr":
        numay = "09"
    elif ay == "oktyabr":
        numay = "10"
    elif ay == "noyabr":
        numay = "11"
    elif ay == "dekabr":
        numay = "12"
    else:
        numay ="Nan"
    
    return f"{numay} {gun}"

In [138]:
def scrape_new_window(href , dic):
    driver.get(href)

    dic["Full_name"].append((driver.find_element(By.XPATH , "//h1[@class = 'product-title']")).text)

    temp = driver.find_element(By.XPATH , "//div[@class = 'product-properties__column']")

    dic["Location"].append((temp.find_element(By.XPATH , "//span[@class = 'product-properties__i-value']")).text)

    dic["Comp_name"].append((temp.find_element(By.TAG_NAME , "a")).text)

    tempp = driver.find_element(By.XPATH , "//div[@class = 'product-price__i product-price__i--bold']")
    
    temppspans = tempp.find_elements(By.TAG_NAME , "")
    temp1 = driver.find_element(By.XPATH , "//div[@class = 'product-info product-info__statistics']")

    temp2 = temp1.find_elements(By.TAG_NAME , "div")

    date = (str(temp2[1].find_element(By.TAG_NAME , "span").text)).split(',')[0]

    if date == "Bugün":
        today = datetime.today()

        dic["date"].append(today.strftime('%m %d'))
        
    elif date == "Dünən":

        yesterday = datetime.today() - timedelta(days=1)

        dic["date"].append(yesterday.strftime('%B %d'))
    else:
        dic["date"].append(azetoengdate(date))

    driver.back()

    return dic


In [139]:
path1 = "https:/tap.az"

driver.get(path1)

time.sleep(5)
try:
    category = driver.find_element(By.XPATH , "//a[@data-for = 'consumer-electronics']")

    driver.get(category.get_attribute('href'))

    # button = driver.find_element(By.XPATH , "//button[@class = 'filter-dropdown' and @type = 'submit']")

    # button.click()

    dropdown = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'price']")

    dropdown.click()
    
    minprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_lower']")

    maxprice = driver.find_element(By.XPATH , "//input[@class = 'string required form-control' and @id = 'price_upper']")

    minprice.send_keys("1000")

    maxprice.send_keys("2200")

    time.sleep(5)

    categories = driver.find_element(By.XPATH , "//div[@class = 'subcategories-inner']")

    uls = categories.find_elements(By.TAG_NAME , 'ul')

    for ul in uls:
        
        lis = ul.find_elements(By.TAG_NAME , 'li')

        for li in lis:

            if "noutbuk" in str(li.find_element(By.TAG_NAME , 'a').get_attribute('href')).lower():

                href = li.find_element(By.TAG_NAME , 'a').get_attribute('href')

                break
    
    driver.get(href)

    dropdown_new = driver.find_element(By.XPATH , "//div[@class = 'filter-dropdown' and @id = 'new']")

    dropdown_new.click()

    dropdown_new.find_element(By.TAG_NAME,"ul").find_elements(By.TAG_NAME , 'li')[1].click()
    
    time.sleep(10)

    product_columns = driver.find_element(By.XPATH ,"//div[@class = 'js-endless-container products endless-products']" )

    time.sleep(3)

    products = product_columns.find_elements(By.XPATH, "./div")

    dic_comp = {}

    dic_comp["Location"] = []

    dic_comp["date"] = []

    dic_comp["Link"] = []

    dic_comp["price"] = []

    dic_comp["Comp_name"] = []

    dic_comp["Full_name"] = []

    dic_comp["Currency"] = []

    i = 0 

    for product in products:

        time.sleep(3)
        
        a = product.find_element(By.TAG_NAME , 'a')

        dic_comp["Link"].append(a.get_attribute("href"))

        dic_comp["Currency"].append(a.find_element(By.XPATH , "//span[@class = 'price-cur']").text)

        dic_comp = scrape_new_window(a.get_attribute("href") , dic_comp)

        i +=1

        if i == 5:
            break
        
finally:
    driver.close()



products-i rounded bumped products-shop
products-i rounded bumped products-shop
products-i rounded bumped products-shop
products-i rounded bumped


KeyboardInterrupt: 

In [140]:
print(dic_comp)

{'Location': ['Bakı', 'Bakı', 'Bakı', 'Bakı'], 'date': ['03 31', '03 31', '03 31', '03 31'], 'Link': ['https://tap.az/elanlar/elektronika/noutbuklar/40871060', 'https://tap.az/elanlar/elektronika/noutbuklar/40871086', 'https://tap.az/elanlar/elektronika/noutbuklar/40664540', 'https://tap.az/elanlar/elektronika/noutbuklar/40827648'], 'Price': ['1 550', '1 550', '1 550', '1 550'], 'Comp_name': ['Lenovo', 'Lenovo', 'Asus', 'Acer'], 'Full_name': ['Lenovo Thinkpad T14', 'Lenovo Thinkpad T14', 'Noutbuk "Asus Rog Strix"', 'Noutbuk "Acer Nitro 5"'], 'Currency': ['AZN', 'AZN', 'AZN', 'AZN']}


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x000000010518441c chromedriver + 4326428
1   chromedriver                        0x000000010517c8e4 chromedriver + 4294884
2   chromedriver                        0x0000000104da7eec chromedriver + 278252
3   chromedriver                        0x0000000104ddebcc chromedriver + 502732
4   chromedriver                        0x0000000104e07de4 chromedriver + 671204
5   chromedriver                        0x0000000104e03694 chromedriver + 652948
6   chromedriver                        0x0000000104e02a7c chromedriver + 649852
7   chromedriver                        0x0000000104d7a1dc chromedriver + 90588
8   chromedriver                        0x00000001051479b8 chromedriver + 4078008
9   chromedriver                        0x000000010514c770 chromedriver + 4097904
10  chromedriver                        0x000000010512e55c chromedriver + 3974492
11  chromedriver                      